Este agarra las imagenes de spectrogramas en spect/ y crea la matriz de grayscale (la estandariza, not forget!! see info.md) y la carga a un CSV (despues vemos donde)

In [ ]:
import os, sys
import pandas as pd
import librosa as lbrs
import numpy as np
from PIL import Image

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import util

In [ ]:
# Aux Functions
def load_TrainDataFrame(spects_source_dir, spects_meta_df, meta_csv_path): 
    ''' Takes the spectrograms from the spects_source_dir folder, and gets the images into a matrix form, 
    with the grayscale of all the pixels, so they can be later made tensors of compatible CNN training input. 
    It loads the matrices pixel info into a CSV so they can then be read & extracted, easily reconverted into 
    a feedable matrix without having to re-run this notebook again. The target CSV (at database/meta/train_data.csv) 
    should have a row for each sample, with the columns class_id, and then all the pixel elements of the spectrogram.

    Args:
        spects_source_dir (str): Directory where the spectrogram images are stored in .png format.
        spects_meta_csv (pd.DataFrame): DataFrame for each spectrogram, with columns 'filename' and 'class_id'.
        meta_csv_path (str): Path to the CSV file where the final data will be stored, with the matrix elements and the class_id labeling each sample
    '''
    data_rows = []
    spects_meta_df = spects_meta_df.dropna(subset=['filename', 'class_id'])

    for _, row in spects_meta_df.iterrows():
        filename = row['filename']
        class_id = row['class_id']

        image_path = os.path.join(spects_source_dir, 'dev', filename + '.png')
        if not os.path.exists(image_path):
            continue

        img = Image.open(image_path).convert('L')  # Ensure grayscale
        
        expected_shape = (313, 224)
        if img.size != expected_shape:
            raise ValueError(f"Unexpected image size: {img.size} in file {image_path}. Expected {expected_shape}.")
        pixels = np.array(img).flatten()

        data_row = [class_id] + pixels.tolist()
        data_rows.append(data_row)

    if not data_rows:
        raise ValueError("No spectrograms were loaded. Check paths and metadata consistency.")

    n_pixels = len(data_rows[0]) - 1
    columns = ['label'] + [f'pixel_{i}' for i in range(n_pixels)]
    df = pd.DataFrame(data_rows, columns=columns)

    # Save to CSV
    df.to_csv(meta_csv_path, index=False)
    print(f"Data saved to {meta_csv_path} with {len(df)} samples and {n_pixels} pixels each.")

In [ ]:
spect_dir = os.path.join('..', 'database', 'spect') # Has the spectrogram pngs
spects_df = pd.read_csv(os.path.join('..', 'database', 'meta', 'final_spects.csv')) # Has the spectrogram metadata ('filename', 'class_id')
data_csv_path = os.path.join('..', 'database', 'meta', 'train_data.csv') # path where the final data_csv will be saved, with the grayscale pixel matrices and class ids

In [ ]:
print("Loading DataFrame...")
train_df = load_TrainDataFrame(spect_dir, spects_df, data_csv_path)

print("Loading CSV...")
train_df.to_csv(data_csv_path, index=False)